Retrieve models from BioModels.
Adapted from

In [1]:
"""
SG : NOT WORKING !!! See Test
https://bioservices.readthedocs.io/en/main/_modules/bioservices/biomodels.html
"""

import os
import subprocess
import sys
import zipfile as z
import requests

try:
    from bioservices import BioModels
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "bioservices"])
    from bioservices import BioModels

# Set custom directory for storing the models (outside of current directory)
modelsDir = '../models'  # Change this path as needed

# Create models directory if it doesn't exist
if not os.path.exists(modelsDir):
    os.makedirs(modelsDir)

s = BioModels()

def get_filtered_models() -> list:
    query_parts_dc = {
        'mode': '*:*',
        'species': 'TAXONOMY:9606',
        'curation_status': 'curationstatus:"Manually curated"',
        'formats': 'modelformat:"SBML"',
        'kw': 'submitter_keywords:"Immuno-oncology"'
    }

    query_parts = [value for value in query_parts_dc.values() if value]
    query = " AND ".join(query_parts)

    print(f"Generated query: {query}")

    try:
        # Perform the search with the generated query
        search_results = s.search(query)

        if not search_results or 'models' not in search_results or not search_results['models']:
            print("No models found.")
            return []

        models_and_metadata = []
        for model in search_results['models']:
            model_id = model['id']
            metadata = {
                'name': model['name'],
                'organism': model.get('organism', ''),
                'curation_status': model.get('curationstatus', ''),
                'go_terms': model.get('go', [])
            }
            models_and_metadata.append({'model_id': model_id, 'metadata': metadata})

        return models_and_metadata

    except Exception as e:
        print(f"Error during search: {e}")
        return []

def download_biomodels(directory: str, models_and_metadata: list, num_per_download=100):
    """ Downloads SBML models along with their metadata from BioModels. """
    if num_per_download > 100:
        raise ValueError("Maximum number of models that can be downloaded at a time is 100")

    num_downloads = len(models_and_metadata) // num_per_download + (len(models_and_metadata) % num_per_download > 0)

    filenames = []
    for i in range(num_downloads):
        start_index = i * num_per_download
        end_index = min((i + 1) * num_per_download, len(models_and_metadata))
        batch_models = models_and_metadata[start_index:end_index]

        fname = os.path.join(directory, f"Biomodels_{start_index + 1}_{end_index}.zip")
        filenames.append(fname)

        if os.path.isfile(fname):
            os.remove(fname)

        # Download each model in the batch
        with z.ZipFile(fname, 'w') as zf:
            for model in batch_models:
                model_id = model['model_id']
                try:
                    # Use BioModels API to get the model file
                    model_content = s.get_model_download(model_id)
                    zf.writestr(f"models/{model_id}.xml", model_content.encode('utf-8'))  # Save in models folder inside zip
                    print(f"Downloaded model {model_id} to zip.")
                except Exception as e:
                    print(f"Error on model {model_id}: {e}")

    # Consolidate zips
    biomodels_zip = os.path.join(directory, "biomodels.zip")
    with z.ZipFile(biomodels_zip, 'w') as consolidated_zip:
        for fname in filenames:
            with z.ZipFile(fname, 'r') as zf:
                for n in zf.namelist():
                    consolidated_zip.writestr(n, zf.open(n).read())

    # Clean up individual zips
    for fname in filenames:
        os.remove(fname)

    return biomodels_zip

if __name__ == "__main__":
    # Set to True to actually perform the download
    ACTIVATE_DOWNLOAD = True

    if ACTIVATE_DOWNLOAD:
        models_and_metadata = get_filtered_models()
        if models_and_metadata:
            final_zip = download_biomodels(modelsDir, models_and_metadata)
            print(f"All models saved in {final_zip}")


WARNING [bioservices.BioModels:130]:  The URL (https://www.ebi.ac.uk/biomodels) provided cannot be reached.
INFO    [bioservices.BioModels:363]:  Initialising BioModels service (REST)


Generated query: *:* AND TAXONOMY:9606 AND curationstatus:"Manually curated" AND modelformat:"SBML" AND submitter_keywords:"Immuno-oncology"
Error downloading batch 1-10: sequence item 0: expected str instance, dict found
Error consolidating ../models/Biomodels_1-10.zip: [Errno 2] No such file or directory: '../models/Biomodels_1-10.zip'
All models consolidated into ../models/biomodels.zip
All models saved in ../models/biomodels.zip
